In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import svm
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

/Users/laurencewilliams/opt/anaconda3/envs/scover/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
df = pd.read_csv("data.csv")

In [4]:
data = df.sort_values(by = 'Date', ascending = True)

In [5]:
df["region"].replace(' ', '_', regex=True, inplace=True)

In [6]:
df = df[~df['region'].isin(["Ile-de-France","(not_set)","Occitanie","England","Bogota","Bavaria","North_Rhine-Westphalia","Porto_District","Santiago_Metropolitan_Region","Morelos","Melilla","Scotland","Andorra_la_Vella","Zulia","Prague","Lombardy","Hessen","New_York","Capital_District","North_Holland","Occitania","Azuay","Cartago_Province","Utrecht","New_Hampshire","Pays_de_la_Loire","Georgia","Santa_Fe_Province","Buenos_Aires","Rio_Grande","Vienna"])]

In [7]:
df2 = df[[ 'categorySlug', 'region', 'RC', 'Comercio', 'sale']]

In [8]:
df3=df2.dropna()

In [9]:
df3

categorySlug               region   RC  Comercio  \
0     muebles-decoracion-y-menaje-hogar            Catalonia  1.0       0.0   
1                   servicios-generales     Balearic_Islands  1.0       0.0   
2                arte-y-entretenimiento            Catalonia  1.0       0.0   
3                   servicios-generales               Madrid  1.0       0.0   
4               construccion-y-reformas              Galicia  1.0       0.0   
...                                 ...                  ...  ...       ...   
8057                    deportes-y-ocio  Valencian_Community  1.0       0.0   
8058                    deportes-y-ocio            Andalusia  1.0       0.0   
8059                    deportes-y-ocio               Madrid  1.0       0.0   
8060                    deportes-y-ocio              Navarre  1.0       0.0   
8061                    deportes-y-ocio     Balearic_Islands  1.0       0.0   

      sale  
0        0  
1        0  
2        0  
3        0  
4        0  
...    ...  
8057     0  
8058     0  
8059     0  
8060     0  
8061     0  

[7289 rows x 5 columns]

# Feature engineering:

In [55]:
def get_dummies(df, varlist):
    for var in varlist:
        df_slice = pd.get_dummies(df[var])
        df = pd.concat([df.drop(var, axis =1), df_slice], axis =1)
    return df

In [56]:
categorical_list = ["categorySlug", "region"]

In [57]:
categorical_list = ["categorySlug","region"]
X = df3.drop(columns='sale')
X = get_dummies(X, categorical_list)
y = df3['sale']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [59]:
df3['sale'].value_counts()

0    6456
1     833
Name: sale, dtype: int64

In [60]:
from imblearn.combine import SMOTEENN 
from imblearn.combine import SMOTETomek

oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

0    4531
1    2718
Name: sale, dtype: int64

In [38]:
model = RandomForestClassifier()

In [61]:
model.fit(X_train_b, np.ravel(y_train_b)).score(X_test, np.ravel(y_test))

0.7590306355738454

In [63]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [64]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [65]:
rf_random.fit(X_train_b, y_train_b)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.4min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [66]:
print('Random Forest classifier best paramas')
print(rf_random.best_score_)
print(rf_random.best_params_)

Random Forest classifier best paramas
0.7089286497135356
{'n_estimators': 1800, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': False}


In [67]:
rf_optimal = RandomForestClassifier(n_estimators = 1800, min_samples_split = 10,
                                    min_samples_leaf = 1, max_features = 'sqrt', max_depth = 30,
                                    bootstrap = False)

In [68]:
rf_optimal.fit(X_train_b, y_train_b)

RandomForestClassifier(bootstrap=False, max_depth=30, max_features='sqrt',
                       min_samples_split=10, n_estimators=1800)

In [70]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions_train = rf_optimal.predict(X_train_b)
predictions_test = rf_optimal.predict(X_test)
print(classification_report(y_test, predictions_test))
print("accuracy score on test:", accuracy_score(y_test, predictions_test))
print("accuracy on train:", accuracy_score(y_train_b, predictions_train))

              precision    recall  f1-score   support

           0       0.90      0.82      0.86      1925
           1       0.19      0.31      0.23       262

    accuracy                           0.76      2187
   macro avg       0.54      0.56      0.55      2187
weighted avg       0.81      0.76      0.78      2187

accuracy score on test: 0.7613168724279835
accuracy on train: 0.7303076286384329
